<h1>Digit Recognization With Tensorflow</h1>

<h2 style="font-size: 20px;">Import Necessary Libraries and packages</h2>

In [1]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.2 MB/s eta 0:00:00


In [2]:
import os
import datetime
import numpy as np
import pandas as pd
import keras_tuner as kt
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from typing import Union, Tuple, Optional
print("tensorflow version: ", tf.__version__)

tensorflow version:  2.9.2


In [3]:
%load_ext tensorboard

<h2 style="font-size: 20px;">Setting The Notebook</h2>

In [4]:
# Setting the graph style
plt.rc('figure', autolayout=True)
plt.rc(
    'axes', titleweight='bold', 
    titlesize=20, labelweight=700,
    labelsize=13
    )
plt.rc('font', size=15)

def set_seeds(seed: int=0):
    """Sets the Seed into order to get the same result on every code run"""
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    os.environ["TF_DETERMINISTIC"] = str(seed)
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"

# Call the set_seeds with default parameter
set_seeds()

try:
    tfu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    stratagy = tf.distribute.TPUStrategy(tfu)
except ValueError:
    stratagy = tf.distribute.get_strategy()
print(f"Number of Accelerator: {stratagy.num_replicas_in_sync}")

Number of Accelerator: 1


In [5]:
train_nrow = 42000
test_nrow = 28000
training_size = train_nrow - test_nrow
BATCH_SIZE_PER_REPLICA = 32
batch_size = BATCH_SIZE_PER_REPLICA * stratagy.num_replicas_in_sync
steps_per_epoch = training_size // batch_size
validation_steps = test_nrow // batch_size

<h2 style="font-size: 20px;">Above Dataset</h2>
<p style="font-size: 15px;">
The data files train.csv and test.csv contain gray-scale images of hand-drawn digits, from zero through nine.

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

Each pixel column in the training set has a name like pixelx, where x is an integer between 0 and 783, inclusive. To locate this pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27, inclusive. Then pixelx is located on row i and column j of a 28 x 28 matrix, (indexing by zero).
</p>

<h2 style="font-size: 20px;">Load Csv File Into Pandas DataFrame</h2>

In [ ]:
try:
    from google.colab import drive
    # Mount google files
    drive.mount('/gdrive')

    # list the files from drive
    folder_path = '../gdrive/MyDrive/Datasets/MNIST_data'
    
except ImportError:
    folder_path = '../input/digit-recognizer'

os.listdir(folder_path)

In [ ]:
# Data Path
train_data_path = folder_path + '/train.csv'
test_data_path = folder_path + '/test.csv'

# Loading dataset into pandas 
train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)

<h2 style="font-size: 20px;">View The Dataframe</h2>

In [ ]:
# First Rows in the train data
print(train_df.shape)
train_df.head()

In [ ]:
# First Rows in the test data
print(test_df.shape)
test_df.head()

<h2 style="font-size: 20px;">Train Data Split Into Train and Validation data</h2>

In [ ]:
mask = np.random.randn(len(train_df)) > 0.8
train_data = train_df[mask]
valid_data = train_df[~mask]
print(f"Train_data shape: {train_data.shape}\nValid_data: {valid_data.shape}")

<h2 style="font-size: 20px;">Create Dataset Generator</h2>

In [ ]:
class DatasetGenerator:
    def __init__(self, batch_size: int = batch_size, shuffle: int = False, epoch: int = 10) -> None:
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.epoch = epoch

    def __call__(self, df: pd.DataFrame) -> any:
        if self.shuffle:
            df_ = tf.constant([
                row[1:].values / 255
                for _, row in df.iterrows()
                ])
            df_ = tf.reshape(df_, [len(df), 28, 28, 1])
            labels = df.iloc[:, 0]
            data = tf.data.Dataset.from_tensor_slices((df_, labels)).batch(self.batch_size).shuffle(1000)
        else:
            df_ = tf.constant([
                row.values.reshape(28, 28) /255
                for _, row in df.iterrows()
                ])
            df_ = tf.reshape(df_, [len(df), 28, 28, 1])
            data = tf.data.Dataset.from_tensor_slices((df_)).batch(self.batch_size)
        return data

<h2 style="font-size: 20px;">Convert The Data To tensor</h2>

In [ ]:
# Change the dataframes to generator 
train_data = DatasetGenerator(shuffle=True, batch_size=batch_size)(train_data)
validation_data = DatasetGenerator(shuffle=True, batch_size=batch_size)(valid_data)
test_dataset = DatasetGenerator()(test_df)
train_data

<h2 style="font-size: 20px;">Exploring Datasets</h2>

In [ ]:
# Exploring One of the image in the train_dataset generator
img, label = next(iter(train_data))

In [ ]:
print("image Datatype: ", type(img[0]))
print("label data type: ", type(label[1]))
print("label: ", set(label.numpy()))

In [ ]:
# For exploratory purpose change the image to numpy array
np_img = np.array(img)
IMG_SHAPE = np_img.shape
print("Image shape: ", IMG_SHAPE)

<h2 style="font-size: 20px;">Digits Visualization</h2>

In [ ]:
# Visualizing of the one image in the test_dataset
img = next(iter(test_dataset))
cmap = plt.get_cmap('magma')
plt.imshow(tf.squeeze(img[0]), cmap=cmap)
plt.axis('off')
plt.show()


In [ ]:
# Create Subplot instance
fig, ax = plt.subplots(figsize=(8, 8))

# Number of columns and rows
NROW = 3
NCOL = 3

# loop over the range of NROW * NCOL starting from 1
for i in range(1, NROW * NROW + 1):
    fig.add_subplot(NROW, NCOL, i)

    # Get the image from the generated index
    img, label = next(iter(train_data))

    # Plot it
    plt.imshow(tf.squeeze(img[0]), 
        cmap=cmap
        )
    plt.axis('off')
    plt.title(label[0].numpy())
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_axis_off()

<h2 style="font-size: 20px;">Modeling</h2>

In [ ]:

def build_model(input_shape, eager: bool = False):
    """Create a Keras Sequential and complie it"""
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))

    model.add(layers.Dense(units=256, activation='relu'))
    # model.add(layers.BatchNormalization())
    model.add(layers.Dropout(rate=0.5))

    model.add(layers.Dense(units=256, activation='relu'))
    # model.add(layers.BatchNormalization())
    model.add(layers.Dropout(rate=0.5))

    model.add(layers.Dense(units=256, activation='relu'))
    # model.add(layers.BatchNormalization())
    model.add(layers.Dropout(rate=0.2))

    # Add a output layer
    model.add(layers.Dense(units=10, activation='softmax'))

    if eager:
        model.compile(
            optimizer='adam',
            loss='SparseCategoricalCrossentropy',
            metrics=['acc'],
            run_eagerly=True

        )
    else:
        model.compile(
            optimizer='adam',
            loss='SparseCategoricalCrossentropy',
            metrics=['accuracy'],
        )
    return model

# Model instance
model = build_model(IMG_SHAPE[1:], eager=True)

# Plot the model
dot_image_file = '/tmp/model_1.png'
keras.utils.plot_model(model, to_file=dot_image_file, show_shapes=True)

In [ ]:
# Initialize the EarlyStopping class: early_stopping
early_stopping = callbacks.EarlyStopping(
    patience=1,
    min_delta=0.01,
    restore_best_weights=True
    
)

Callback to save the Keras model or model weights at some frequency

In [ ]:
checkpoint_filepath = "/tmp/checkpoints"

model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

Reduce learning rate when a metric has stopped improving

In [ ]:
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=0, min_lr=0.001, mode='auto', verbose=1)

Initialize the tensorboard for debuging and visualizing the model

In [ ]:
tensorboard = callbacks.TensorBoard("/tmp/tb_logs")

In [ ]:

# Fit the model
with stratagy.scope():
    model = build_model(IMG_SHAPE[1:])

history = model.fit(
    train_data, 
    validation_data=validation_data, 
    epochs=15,
    callbacks=[early_stopping, reduce_lr],
    batch_size=batch_size,
    validation_steps=validation_steps, 
    )

In [ ]:
# Training without early_stopping callback
# Fit the model
with stratagy.scope():
    model = build_model(IMG_SHAPE[1:])

history = model.fit(
    train_data, 
    validation_data=validation_data,
    epochs=20,
    callbacks=[
            #    model_checkpoint_callback, 
                # reduce_lr, 
            #    tensorboard
               ],
    batch_size=batch_size,
    validation_steps=validation_steps
    )

<h2 style="font-size: 20px;">Model Evaluation</h2>

In [ ]:
evalutate = model.evaluate(validation_data)
print(f"Loss: {round(evalutate[0], 4)} - Accurary: {round(evalutate[1], 3)}%")

In [ ]:
# Construct a dataframe with loss and accuracy score
history_df = pd.DataFrame(history.history)
history_df.head()

In [ ]:
def metric_vis(train: str, val: str, ax: plt.axis) -> None:

    fig1 = history_df[train].plot(c='pink', ax=ax, marker="x")
    fig2 = history_df[val].plot(c='skyblue', ax=ax, marker="o")
    ax.set_ylabel(f"{train} Score")
    ax.set_xlabel("Epoch/Iteration")
    ax.set_title(f"Train And Validation {train}".title())
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.legend([f'{train}', f"{val}"])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
metric_vis("loss", 'val_loss', ax=ax[0])
metric_vis("accuracy", 'val_accuracy', ax=ax[1])

In [ ]:
%tensorboard --logdir /tmp/tb_logs

## **Hypeparameter Tuning**

In [ ]:
def model_builder(hp: kt.HyperParameters) -> keras.Sequential:
    """
    Create a Keras Sequential and complie it
    :param hp for hypeparameter values:
    :return tf.keras.Sequential model:
    """
    # Construct a model
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=IMG_SHAPE[1:]))

    # Tune the number of units
    # Ranging from 32 to 512
    hp_units = hp.Int("units", min_value=32, max_value=512, step=32)
    
    # Tune the number of drop out rate
    # Rate will be in range of 0.1 to 0.5
    hp_rate = hp.Float('rate', min_value=0.1, max_value=1)

    # Tune the activation functions
    hp_act = hp.Choice('activation', values=[
                'relu', 'elu', 'gelu',
                'linear', 'selu', 'swish',
                'tanh'
                ])

    # *** looking for prefect number of layers ***
    hidden_layers = hp.Choice("hidden_layers", values=["two", "three", "four", "six"])

    if hidden_layers == "two":
        with hp.conditional_scope("hidden_layers", ["two"]):
            # first Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())

            # second Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())
    
    if hidden_layers == "three":
        with hp.conditional_scope("hidden_layers", ["three"]):
            # first Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())

            # second Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())

            # three Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())
    
    if hidden_layers == "four":
        with hp.conditional_scope("hidden_layers", ["four"]):
            # first Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())

            # second Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())

            # third Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())

            # fourth Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())
    
    if hidden_layers == "six":
        with hp.conditional_scope("hidden_layers", ["six"]):
            # first Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())

            # second Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())

            # third Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())

            # fourth Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())

            # fifth Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())

            # sixth Hidden layer
            model.add(layers.Dense(units=hp_units, activation=hp_act))
            # Drop out layer
            model.add(layers.Dropout(rate=hp_rate))
            # Normalization data
            model.add(layers.BatchNormalization())

    # Add a output layer
    model.add(layers.Dense(units=10, activation='softmax'))

    
    # Compile the model
    # Tune the learning rate for optimizer
    # Ranging from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-3, 1e-4, 1e-2, ])

    # **** looking for best optimizer *****

    hp_optimizer = hp.Choice("optimizer", values=[
                        "Adam", "RMSprop", 
                        "Adamax", "Nadam",
                        "SGD"
                        ])

    if hp_optimizer == "Adam":
        with hp.conditional_scope("optimizer", ["Adam"]):
            model.compile(
                optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=["accuracy"]
            )

    if hp_optimizer == "RMSprop":
        with hp.conditional_scope("optimizer", ["RMSprop"]):
            model.compile(
                optimizer=keras.optimizers.RMSprop(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=["accuracy"]
            )

    if hp_optimizer == "Adamax":
        with hp.conditional_scope("optimizer", ["Adamax"]):
            model.compile(
                optimizer=keras.optimizers.Adamax(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=["accuracy"]
            )

    if hp_optimizer == "Nadam":
        with hp.conditional_scope("optimizer", ["Nadam"]):
            model.compile(
                optimizer=keras.optimizers.Adamax(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=["accuracy"]
            )

    if hp_optimizer == "SGD":
        with hp.conditional_scope("optimizer", ["SGD"]):
            model.compile(
                optimizer=keras.optimizers.Adamax(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=["accuracy"]
            )

    return model

In [ ]:
# Instantiate tuner and preform hypertuning
tuner = kt.Hyperband(
    model_builder,
    objective="val_accuracy",
    max_epochs=10,
    directory="/tmp/hypeparameters",
    project_name="intro_to_hypetuning"
)

In [ ]:
# Initialize the earlystopping instance
early_stopping = callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

# tuner.search(
#     train_data,
#     validation_data=validation_data,
#     epochs=8,
#     callbacks=[early_stopping]
# )

In [ ]:
# tuner.results_summary()

In [ ]:
# best_hps = tuner.get_best_hyperparameters(num_trials=5)
# print(f"""
# The hyperparameter search is complete. The optimal number of units in the first densely-connected
# layer is {best_hps[0].get('units')} and the optimal learning rate for the optimizer
# is {best_hps[0].get('learning_rate')}.
# """)

In [ ]:
# built_model = tuner.hypermodel.build(best_hps[0])

# # Fit the best model from the hypetuning
# built_model.fit(
#     train_data, 
#     validation_data=validation_data,
#     epochs=20,
#     callbacks=[
#                model_checkpoint_callback, 
#                reduce_lr, 
#                tensorboard
#                ],
#     # Made same improve with these two code
#     batch_size=batch_size,
#     validation_steps=validation_steps
#     )

## **Building A Model with Convolutional Layers**

In [ ]:
# Using one since the images are in gray scale.
input_shape = [64, 28, 28, 1]
input_shape

In [ ]:
class ConvModel(keras.Model):

    # Constructor .....
    def __init__(self):
        super(ConvModel, self).__init__()

        # Base Blocks ..........................

        # First Block .....
        self.first_conv2d = layers.Conv2D(
            filters=1024,
            activation='leaky_relu',
            kernel_size=3,
            input_shape = input_shape,
            name='First_conv2D'
            )
        self.max_pool_1 = layers.MaxPooling2D(name='MaxPooling2D_1')
        # Drop same pixels ....
        self.dropout_1 = layers.Dropout(.5, name='Dropout_1')

        # Second Block .....
        self.second_conv2d = layers.Conv2D(
            filters=512,
            activation='relu',
            kernel_size=3,
            name='Second_conv2D'
           )
        self.max_pool_2 = layers.MaxPooling2D(name='MaxPooling2D_2')
        # Drop same pixels ....
        self.dropout_2 = layers.Dropout(.5, name='Dropout_2')

        # Third Block .....
        self.third_conv2d = layers.Conv2D(
            filters=256,
            activation='leaky_relu',
            kernel_size=3,
            name='Third_conv2D'
           )
        # self.avg_pooling_3 = layers.AveragePooling2D(name='AveragePooling2D_1')
        # Drop same pixels ....
        self.dropout_3 = layers.Dropout(.5, name='Dropout_3')


        # Forth Block .....
        self.forth_conv2d = layers.Conv2D(
            filters=128,
            activation='relu',
            kernel_size=3,
            name='forth_conv2D'
           )
        # self.max_pool_4 = layers.MaxPooling2D(name='MaxPooling2D_4')
        # Drop same pixels ....
        self.dropout_4 = layers.Dropout(.6, name='Dropout_4')


        # Head Block ............................

        # Data Flatten Layer .....
        self.flatten = layers.Flatten()

        # Hidden layer ...
        self.first_hidden_dense = layers.Dense(
            units=128, activation='leaky_relu', name='Hidden_layer_1'
            )

        # Output layer .....
        self._output = layers.Dense(
            units=10, activation='softmax', name='Output_layer'
            )


    def call(self, X) -> tf.Tensor:
        # Base Blocks ..........................

        # First Block .....
        first_conv2d = self.first_conv2d(X)
        max_pool_1 = self.max_pool_1(first_conv2d)
        # Drop same pixels ....
        dropout_1 = self.dropout_1(max_pool_1)

        # Second Block .....
        second_conv2d = self.second_conv2d(dropout_1)
        max_pool_2 = self.max_pool_2(second_conv2d)
        # Drop same pixels ....
        dropout_2 = self.dropout_2(max_pool_2)

        # Third Block .....
        third_conv2d = self.third_conv2d(dropout_2)
        # avg_pool_1 = self.avg_pooling_3(third_conv2d)
        # Drop same pixels ....
        dropout_3 = self.dropout_3(third_conv2d)

        # Forth Block .....
        forth_conv2d = self.forth_conv2d(dropout_3)
        # max_pool_4 = self.max_pool_4(forth_conv2d)
        # Drop same pixels ....
        dropout_4 = self.dropout_4(forth_conv2d)



        # Head Block ............................

        # Data Flatten Layer .....
        flatten = self.flatten(dropout_4)

        # Hidden layers .....
        first_hidden_dense = self.first_hidden_dense(flatten)

        # Output layer .....
        _output = self._output(first_hidden_dense)
        
        return _output


In [ ]:
with stratagy.scope():
    # Instantiate the model object .....
    conv_model = ConvModel()

    # Build the model .....
    conv_model.build(input_shape)

    # Get the Model summary ...
    conv_model.summary()

    # Compile the model ...
    conv_model.compile(
            optimizer='adam',
            loss=keras.losses.SparseCategoricalCrossentropy(),
            metrics=['accuracy'],
        )

# Fit the Conv model ....
conv_model.fit(
    train_data, 
    validation_data=validation_data, 
    epochs=20,
    # callbacks=[
    #            model_checkpoint_callback, 
    #            reduce_lr, 
    #            tensorboard
    #            ],
    batch_size=batch_size
    )

## Fine Tune Convalutional Model

In [ ]:
def build_cov_model(hp: kt.HyperParameters):
    # Tune the convolutional model filters.
    filters = hp.Choice(name='filter', values=[64, 128, 256, 512])

    # Instantite the Sequential model.
    conv_model = keras.Sequential()

    # The base.
    # Adding the First convolutional layer.
    conv_model.add(layers.Conv2D(
        filters=128,
        kernel_size=3,
        activation='relu',
        input_shape=(28, 28, 1)
    ))

    # The Head.
    # Flatten the data.
    conv_model.add(layers.Flatten())

    # Adding the First Dense layer.
    conv_model.add(layers.Dense(
        units=128,
        activation='softmax'
    ))

    # Compile the model.
    conv_model.compile(
        optimizer='rmsprop',
        loss='sparse_categorical_crossentropy'
    )

    return conv_model


In [ ]:
# Initialize the earlystopping instance
early_stopping = callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

# Instantiate tuner and preform hypertuning
conv_tuner = kt.RandomSearch(
    build_cov_model,
    objective="val_accuracy",
    # max_epochs=10,
    directory="/tmp/convolutional_model_hypeparameters",
    project_name="tuning_convolutional_model"
)

# Initialize the earlystopping instance
early_stopping = callbacks.EarlyStopping(monitor='val_accuracy')

conv_tuner.search(
    train_data,
    # validation_data=validation_data,
    epochs=8,
    callbacks=[early_stopping]
)

<h2 style="font-size: 20px;">Submission</h2>

In [ ]:
# Make predictions
prob  = conv_model.predict(test_dataset)
# Get the highest value from probability
pred = tf.argmax(prob, 1)

In [ ]:
submission=pd.read_csv(folder_path + '/sample_submission.csv')
submission['Label']=pred
submission.to_csv('prediction20_conv.csv', index=False)